# LASSO

We wish to recover a sparse vector $x \in \mathbf{R}^n$ from measurements $y \in \mathbf{R}^m$. Our measurement model tells us that

$$
y = Ax + v,
$$

where $A \in \mathbf{R}^{m \times n}$ is a known matrix and $v \in \mathbf{R}^m$ is unknown measurement error. The entries of $v$ are drawn IID from the distribution $\mathcal{N}(0, \sigma^2)$.

We can first try to recover $x$ by solving the optimization problem

\begin{array}{ll} \text{minimize} & ||Ax - y||^2_2 + \gamma ||x||^2_2.\\
\end{array}

This problem is called ridge regression.

The code below defines $n$, $m$, $A$, $x$, and $y$. Use CVXPY to estimate $x$ from $y$ using ridge regression. Try multiple values of $\gamma$. Use the plotting code to compare the estimated $x$ with the true $x$.

A more successful approach is to solve the LASSO problem

\begin{array}{ll} \text{minimize} & ||Ax - y||^2_2 + \gamma \|x\|_1.\\
\end{array}

How many measurements $m$ are needed to find an accurate $x$ with ridge regression? How about with the LASSO?


In [ ]:
# Ridge regression vs. LASSO to estimate sparse x.
import numpy as np
import scipy.sparse as sp

np.random.seed(1)

n = 200
m = 50
true_x = 100 * sp.rand(n, 1, 0.1).todense()
A = np.random.randn(m, n)
sigma = 1
v = np.random.normal(0, sigma, (m, 1))
y = A * true_x + v


In [ ]:
import cvxpy as cp

# Construct the problem.
x = cp.Variable((n, 1))
# TODO: your code here

# Ridge regression
ridge = []
lasso = []
for gamma in np.arange(0, 1, 0.1):
    # gamma = 0.5
    objective = cp.Minimize(
        cp.square(cp.norm(A @ x - y, p=2)) + gamma * cp.square(cp.norm(x, p=2))
    )
    prob = cp.Problem(objective)
    prob.solve()
    ridge.append(x.value)

# Lasso
for gamma in np.arange(0, 1, 0.1):
    # gamma = 0.5
    objective = cp.Minimize(cp.square(cp.norm(A @ x - y, p=2)) + gamma * cp.norm(x, p=1))
    prob = cp.Problem(objective)
    prob.solve()
    print(gamma)
    lasso.append(x.value)

for i in range(len(ridge)):
    print(np.linalg.norm(true_x - ridge[i]), np.linalg.norm(true_x - lasso[i]))

In [ ]:
x.value

In [ ]:
np.linalg.norm(true_x - x.value)

In [ ]:
np.sqrt(sum(np.power((true_x - x.value), 2)))

In [ ]:
sum(np.power((true_x - x.value), 2)) / len(true_x)

In [ ]:
# Plot estimate of x against true x.
import matplotlib.pyplot as plt
plt.plot(range(n), true_x, label="true x")
plt.plot(range(n), x.value, label="estimated x")
plt.plot(range(n), np.abs(true_x - x.value), label="error")
plt.legend(loc="upper right")
plt.show()


In [ ]:
# Plot estimate of x against true x.
import matplotlib.pyplot as plt

plt.plot(range(n), true_x, label="true x")
plt.plot(range(n), x.value, label="estimated x")
plt.plot(range(n), np.abs(true_x - x.value), label="error")
plt.legend(loc="upper right")
plt.show()


In [ ]:
import cvxpy as cp

np.random.seed(1)

ridge = []
lasso = []
for m in range(1100, 1500, 50):
    n = 200
    true_x = 100 * sp.rand(n, 1, 0.1).todense()
    A = np.random.randn(m, n)
    sigma = 1
    v = np.random.normal(0, sigma, (m, 1))
    y = A * true_x + v

    # Construct the problem.
    x = cp.Variable((n, 1))
    gamma = 0.9

    # Ridge regression
    objective = cp.Minimize(
        cp.square(cp.norm(A @ x - y, p=2)) + gamma * cp.square(cp.norm(x, p=2))
    )
    prob = cp.Problem(objective)
    prob.solve()
    current_ridge = np.linalg.norm(true_x - x.value)
    ridge.append(current_ridge)

    # Lasso
    objective = cp.Minimize(
        cp.square(cp.norm(A @ x - y, p=2)) + gamma * cp.norm(x, p=1)
    )
    prob = cp.Problem(objective)
    prob.solve()
    current_lasso = np.linalg.norm(true_x - x.value)
    lasso.append(current_lasso)

    print(m, f"{current_ridge:.4f}", f"{current_lasso:.4f}")
